In [17]:
#Packages

import json
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [18]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
   # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

## Opening Booking.com

In [19]:
# lets open booking:

dfolder='./downloads'
geko_path='/Users/mikelgallo/repos2/text_mining/Scraping_UN/books/geckodriver'
link='https://www.booking.com/index.es.html'

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

#### Pending - Removing popups

#### Opening Search_box

In [20]:
browser.find_element(by='xpath',value= '//input[@id=":re:"]').click()

#### Inputing destination

In [21]:
place = input('Where do you want to go?')
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

Where do you want to go? Barcelona


#### Opening Dates table

In [22]:
#Use CSS Selector Copy
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
browser.find_element('css selector',css).click()

#### Selecting Dates

In [23]:
#Selecting Dates table and retrieving date information
def extract_dates():
    dates_f = []
    path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
    dates = browser.find_elements('xpath',path)
    for date in dates:
        dates_f.append(date)
    return dates_f

In [24]:
dates_f = extract_dates()

for date in dates_f:
    print(date.get_attribute("data-date"))

2024-02-02
2024-02-03
2024-02-04
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-10
2024-02-11
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-17
2024-02-18
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-24
2024-02-25
2024-02-26
2024-02-27
2024-02-28
2024-02-29
2024-03-01
2024-03-02
2024-03-03
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-08
2024-03-09
2024-03-10
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-16
2024-03-17
2024-03-18
2024-03-19
2024-03-20
2024-03-21
2024-03-22
2024-03-23
2024-03-24
2024-03-25
2024-03-26
2024-03-27
2024-03-28
2024-03-29
2024-03-30
2024-03-31


In [25]:
for date in dates_f:
    if date.get_attribute("data-date") == "2024-02-02":
        time.sleep(1)
        date.click()
    if date.get_attribute("data-date") == "2024-02-04":
        time.sleep(1)
        date.click()
        break

In [11]:
#Returning pages
#for i in range(1,6):
    #if i == 1:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #else:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #browser.find_element(by='xpath',value=x_path).click()
    #time.sleep(2)

#### Inputing Dates 

### Searching

In [26]:
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

#### Retrieve data

### Getting the number of Pages

In [27]:
def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@data-testid="pagination"]//li[@class="b16a89683f"]')
    return(int(a[-1].text))


pages = get_number_pages(browser)

print(pages)

19


#### Creating Click button

In [28]:
#Click next page
x_path='/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
#browser.find_element(by='xpath',value=x_path).click()

In [19]:
len(descriptions_list)

52

#### Dates to look for

In [29]:
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re

num_pages = 3
df = []
wait = WebDriverWait(browser, 4)
for num in range(1, num_pages+1):
    links = []
    hotels_list = []
    prices_list = []
    districts_list = []
    distance_center = []
    ratings_list = []
    comment_list = []
    stars = []
    time.sleep(4)
    #hotel_overview = browser.find_elements('xpath','//div[@class="c066246e13"]')
    hotel_overview = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="c066246e13"]')))
    for hotel in hotel_overview:
        # Retrieve links (URL)
        try:
            link = hotel.find_elements('xpath', ".//div[@class='d6767e681c']//a")
            links.append(link[0].get_attribute('href'))
        except NoSuchElementException:
            links.append(np.nan)
        # Retrieve Hotel names
        try:
            hotels = hotel.find_elements('xpath', './/div[@class="f6431b446c a15b38c233"]')
            hotels_list.append(hotels[0].text)
        except NoSuchElementException:
            hotels_list.append(np.nan)
        # Retrieve Prices
        try:
            prices = hotel.find_elements('xpath', './/span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]')
            prices_list.append(prices[0].text)
        except NoSuchElementException:
            prices_list.append(np.nan)
        #Retrieve Districts
        try:
            districts = hotel.find_elements('xpath', './/span[@class="aee5343fdb def9bc142a"]')
            for i in districts:
                if ', barcelona' in i.text.lower():
                    districts_list.append(i.text)
        except NoSuchElementException:
            districts_list.append(np.nan)
        except:
            districts_list.append(np.nan)
        # Retrieving only center distances
        try:
            distance = hotel.find_elements('xpath', './/div[@class="abf093bdfe ecc6a9ed89"]//span[@class="f419a93f12"]')
            for i in distance:
                if 'centro' in i.text:
                    distance_center.append(i.text)
        except NoSuchElementException:
            distance_center.append(np.nan)
        except:
            distance_center.append(np.nan)
        # Retrieve Ratings
        try:
            ratings = hotel.find_elements('xpath', './/div[@class="a3b8729ab1 d86cee9b25"]')
            ratings_list.append(ratings[0].text)
        except NoSuchElementException:
            ratings_list.append(np.nan)
        # Retrieve comments
        try:
            comentarios = hotel.find_elements('xpath', './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]')
            comment_list.append(comentarios[0].text)
        except:
            comment_list.append(np.nan)
        try:
            elements = hotel.find_elements('xpath', '//div[@class="b3f3c831be"][@aria-label]')
            stars = [element.get_attribute('aria-label') for element in elements]
        except NoSuchElementException:
            stars.append(np.nan)
        except:
            stars.append(np.nan)
    
    for a, b, c, d, e, f, g, h in zip(hotels_list, distance_center, districts_list, prices_list,ratings_list,stars,comment_list, links):
        row_data = {'Hotels': a, 'Distance': b, 'District': c, 'Price': d, 'Rating': e, 'Stars': f, 'Comments': g, 'Link': h}
        print(row_data)
        df.append(row_data)
        
    
    if num == 1:
        wait = WebDriverWait(browser, 5)  # Adjust the timeout as needed
        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"]')))
        next_button.click()
    elif num == num_pages+1:
        break
    else:
        wait = WebDriverWait(browser, 5)  # Adjust the timeout as needed
        next_button = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"]')))
        next_button[1].click()


{'Hotels': 'Room Mate Gerard', 'Distance': 'a 0,7 km del centro', 'District': 'Eixample, Barcelona', 'Price': '€ 396', 'Rating': '8,8', 'Stars': '4 de 5', 'Comments': '2.783 comentarios', 'Link': 'https://www.booking.com/hotel/es/room-mate-gerard.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AsCU9a0GwAIB0gIkYTQ3ZDRlMWYtM2VlYS00ZDVjLWFlMWQtMmNkYWNmM2QwZWRl2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-02-02&checkout=2024-02-04&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=289f8d05cb8202fb&srepoch=1706904254&all_sr_blocks=224211803_339623744_0_2_0&highlighted_blocks=224211803_339623744_0_2_0&matching_block_id=224211803_339623744_0_2_0&sr_pri_blocks=224211803_339623744_0_2_0__39580&from_sustainable_property_sr=1&from=searchresults#hotelTmpl'}
{'Hotels': 'INNSiDE by Meliá Barcelona Apolo', 'Distance': 'a 1,4 km del centro', 'District': 'Sants-Montjuïc, Barc

In [30]:
df_1 = pd.DataFrame(df)
df_1.shape

(75, 8)

In [32]:
df_1['Link'][0]

'https://www.booking.com/hotel/es/room-mate-gerard.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AsCU9a0GwAIB0gIkYTQ3ZDRlMWYtM2VlYS00ZDVjLWFlMWQtMmNkYWNmM2QwZWRl2AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-02-02&checkout=2024-02-04&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=289f8d05cb8202fb&srepoch=1706904254&all_sr_blocks=224211803_339623744_0_2_0&highlighted_blocks=224211803_339623744_0_2_0&matching_block_id=224211803_339623744_0_2_0&sr_pri_blocks=224211803_339623744_0_2_0__39580&from_sustainable_property_sr=1&from=searchresults#hotelTmpl'

In [41]:
descriptions = []
linkey = df_1['Link'][0:2]
for i in linkey:
    URL = f'{i}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Getting Data
    div2 = soup.find('div', {'id': 'hp_hotel_name'})
    div = soup.find('div', {'id': 'property_description_content'})
    
    # Find all p elements with a specific class within the div
    specific_class_h_elements = div2.find_all('h2', class_='d2fee87262 pp-header__title')
    specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
    
    for h,p in zip(specific_class_h_elements,specific_class_p_elements):
            descriptions.append((h.get_text(strip=True),p.get_text(strip=True)))
    
print(descriptions)


[('Room Mate Gerard', "El Room Mate Gerard en Barcelona ofrece alojamiento con piscina exterior de temporada y centro de fitness.\n\nCuenta con terraza. Su ubicación, en el distrito del Eixample, hace que esté cerca de atracciones como el Teatro Tívoli y a 700 metros de la calle Portal de l'Angel.\n\nTodas las habitaciones del hotel ofrecen una decoración sencilla pero elegante. Están equipadas con armario, TV de pantalla plana y aire acondicionado. Algunas habitaciones incluyen balcón. Todos los alojamientos tienen baño privado con ducha y artículos de aseo gratuitos.\n\nSe sirve un desayuno buffet a diario.\n\nEl personal de la recepción 24 horas habla inglés y español y puede ofrecer información durante la estancia. Otras instalaciones incluyen WiFi portátil.\n\nLa Plaza de Cataluña está a 9 minutos a pie del alojamiento. A 13 km se halla el aeropuerto de Barcelona-El Prat."), ('INNSiDE by Meliá Barcelona Apolo', 'El INNSiDE by Meliá Barcelona Apolo está ubicado a los pies de Montju

In [42]:
import concurrent.futures
from tqdm import tqdm

descriptions = []
linkey = df_1['Link'][0:10]

def scrape_description(link):
    URL = f'{link}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Getting Data
    div2 = soup.find('div', {'id': 'hp_hotel_name'})
    div = soup.find('div', {'id': 'property_description_content'})
    
    # Find all p elements with a specific class within the div
    specific_class_h_elements = div2.find_all('h2', class_='d2fee87262 pp-header__title')
    specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
    
    descriptions = []
    for h, p in zip(specific_class_h_elements, specific_class_p_elements):
        descriptions.append((h.get_text(strip=True), p.get_text(strip=True)))
    
    return descriptions

threads = os.cpu_count()
with concurrent.futures.ThreadPoolExecutor(max_workers=threads-2) as executor:
    results = list(tqdm(executor.map(scrape_description, linkey), total=len(linkey), desc="Processing Links"))

# Flatten the results list
descriptions = [item for sublist in results for item in sublist]


Processing Links: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.75it/s]

[('Room Mate Gerard', "El Room Mate Gerard en Barcelona ofrece alojamiento con piscina exterior de temporada y centro de fitness.\n\nCuenta con terraza. Su ubicación, en el distrito del Eixample, hace que esté cerca de atracciones como el Teatro Tívoli y a 700 metros de la calle Portal de l'Angel.\n\nTodas las habitaciones del hotel ofrecen una decoración sencilla pero elegante. Están equipadas con armario, TV de pantalla plana y aire acondicionado. Algunas habitaciones incluyen balcón. Todos los alojamientos tienen baño privado con ducha y artículos de aseo gratuitos.\n\nSe sirve un desayuno buffet a diario.\n\nEl personal de la recepción 24 horas habla inglés y español y puede ofrecer información durante la estancia. Otras instalaciones incluyen WiFi portátil.\n\nLa Plaza de Cataluña está a 9 minutos a pie del alojamiento. A 13 km se halla el aeropuerto de Barcelona-El Prat."), ('INNSiDE by Meliá Barcelona Apolo', 'El INNSiDE by Meliá Barcelona Apolo está ubicado a los pies de Montju

In [192]:
links = [my_dict.get('Link') for my_dict in df]
len(links)

25

In [16]:
descriptions = []
path = '/Users/mikelgallo/repos2/Bookings/mikel_folder'
links = [my_dict.get('Link') for my_dict in df]
for i in links:
    URL = f'{i}'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div = soup.find('div', {'id': 'property_description_content'})

    # Find all p elements with a specific class within the div
    specific_class_p_elements = div.find_all('p', class_='a53cbfa6de b3efd73f69')
    
    for p in specific_class_p_elements:
        descriptions.append(p.get_text(strip=True))

KeyboardInterrupt: 

In [ ]:
df_new['descriptions'] = descriptions

In [195]:
df_new

,Hotels,Ratings,Distance,District,Price,Link,descriptions
0,H10 Casa Mimosa 4* Sup,"9,2","a 1,2 km del centro","Eixample, Barcelona",€ 1.156,https://www.booking.com/hotel/es/h10-casa-mimo...,El H10 Casa Mimosa está situado en un edificio...
1,Sonder Casa Luz,"8,4",a 450 m del centro,"Eixample, Barcelona",€ 1.787,https://www.booking.com/hotel/es/casa-luz-barc...,El Sonder Casa Luz se encuentra en Barcelona y...
2,MERCE APARTMENT FORUM BARCELONA,"7,6","a 5,5 km del centro","Ciutat Vella, Barcelona",€ 662,https://www.booking.com/hotel/es/merce-apartme...,"MERCE APARTMENT FORUM BARCELONA, que cuenta co..."
3,H La Paloma Adults Only,"8,9","a 0,7 km del centro","Gracia, Barcelona",€ 712,https://www.booking.com/hotel/es/h-la-paloma.e...,The H La Paloma is located in the Raval neighb...
4,Boutique Apartments 23 Barcelona,"8,9","a 1,6 km del centro","Sants-Montjuïc, Barcelona",€ 667,https://www.booking.com/hotel/es/23-barcelona....,Boutique Apartments 23 Barcelona está a menos ...
5,Onefam Sants,"7,3","a 3,6 km del centro","Sant Martí, Barcelona",€ 404,https://www.booking.com/hotel/es/hostel-one-sa...,El Onefam Sants es un animado albergue social ...
6,Travelodge Barcelona Poblenou,"8,2","a 2,9 km del centro","Sant Martí, Barcelona",€ 334,https://www.booking.com/hotel/es/travelodge-ba...,Este moderno hotel cuenta con 250 amplias y co...
7,Catalonia Sagrada Familia,"8,3","a 2,6 km del centro","Sant Martí, Barcelona",€ 441,https://www.booking.com/hotel/es/cataloniaarag...,El Catalonia Sagrada Familia se halla a 15 min...
8,Vincci Maritimo,"8,4","a 4,2 km del centro","Sant Martí, Barcelona",€ 522,https://www.booking.com/hotel/es/vinccimaritim...,El Vincci Marítimo se encuentra a 10 minutos a...
9,Vincci Bit,"8,7","a 4,4 km del centro","Sant Martí, Barcelona",€ 483,https://www.booking.com/hotel/es/vincci-bit.es...,Vincci Bit se encuentra a tan solo 10 minutos ...


In [148]:
browser.quit()